In [1]:
import os

from dotenv import load_dotenv
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스

load_dotenv()

# model = ChatOpenAI(model="gpt-4o-mini")
model = ChatOpenAI(model=os.getenv("DEFAULT_MODEL"))

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
# NOTE: invoke 시 어느 시점에 callback 이 실행 되는지?
#       > test_with_message_history.py
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [2]:
 config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 송정헌이야.")],
    config=config,
)

print(response.content)

만나서 반가워요, 송정헌님! 저는 인공지능 챗봇입니다. 무엇을 도와드릴까요?


In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

맞아요, 당신의 이름은 송정헌입니다! 이젠 잊지 않으셨죠?


In [7]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'당신의 이름을 모르는데 어떻게 대답할 수 있죠? 함께 이 대화를 시작하신 걸 환영합니다! 저는 당신이 저에게 질문하고 새로운 것을 배울 수 있도록 여기 있는 인공지능 챗봇입니다. 당신에 대해 더 알려주시겠어요?'

In [8]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'이전 대화 내용을 되돌아보면, 처음에는 인사를 나누고 당신의 이름을 확인했었어요. 당신은 "안녕? 난 송정헌이야"라고 말했고, 그 이후에 "내 이름이 뭐지?" 라고 세 번 물어보셨죠. 그리고 마지막으로 "아까 우리가 무슨 얘기 했지?" 라고 물어보셨어요!'

In [9]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="|")

|당신의| 이름은 송정헌이고|, 성을 고려하면 한국|인일 가능성이 높다고 판단|됩니다.

한국은 매우 다양한| 문화와 역사를 가진 나라|입니다. 

### 음식 문화|
한식은 세계적으로 유명|합니다. 대표적으로 김치|, 비빔밥,| 불고기, 떡|볶이| 등이 있습니다. 특히 김치는| 한국인의 식탁에서 빠|질 수 없는 발효 음식|으로, 그 종류도 매우| 다양합니다.

### 명절|과 전통 행사
한국에는| 설날과 추석 같은| 큰 명절이 있습니다.| 설날에는 차례를 지|내고, 세|배를 하며, 추석|에는 송|편을 만들어 먹습니다.| 이러한| 전통 행사는 가족과 함께하며| 한국 문화의 중요한 부분을 차지|합니다.

### 한류| 문화
|최근|에는 K-pop, K-dr|ama 등 한류 문화가| 전 세계적으로 인기를 끌|고 있습니다. BTS, BLACK|PINK 같은 K-pop 그룹|이나 "사랑|의| 불시착", "킹|덤" 같은 K-drama|가 많은 팬을 보유하고| 있습니다.

### 전통 예술
|한국은 전통 음악,| 춤, 미술 등 다양한 예술|이| 발달|했습니다. 판소리,| 국악, 사물놀이| 같은| 전통 예술은 한국 문화의| 아름다움을 잘 표현합니다|.

이와 같이 한국은 풍부|한 문화와 역사를 자|랑하는 나라입니다. 더| 알고 싶은 것이| 있|으신가요?||